In [96]:
import json
import networkx as nx
import numpy as np
import plotly.graph_objects as go
from networkx.drawing.nx_agraph import graphviz_layout

In [97]:
filename = "example-2.json"

In [98]:
with open(filename) as f:
    data = json.load(f)

In [99]:
graph = nx.Graph(directed=True)
graph.add_nodes_from(map(lambda scene: scene['scene_id'], data['scenes']))
graph.add_edges_from(
    (choice['next_scene'], scene['scene_id'])
    for scene in data['scenes']
    for choice in scene['choices']
)

In [100]:
# Генерируем координаты вершин графа
pos = graphviz_layout(graph, prog='dot', args="-Grankdir=LR")

coords = np.array(list(pos.values()))


# Извлекаем информацию о положении вершин и рёбер графа
node_x = coords[:, 0] # Заполняем списки вершин
node_y = coords[:, 1]
node_text = list(graph.nodes())
edge_x, edge_y = [], []

for edge in graph.edges():
  n_1 = node_text.index(edge[0])
  n_2 = node_text.index(edge[1])
  edge_x.extend([node_x[n_1], node_x[n_2], None])
  edge_y.extend([node_y[n_1], node_y[n_2], None])

print(edge_x, edge_y)

[np.float64(27.0), np.float64(188.9), None, np.float64(27.0), np.float64(188.9), None, np.float64(188.9), np.float64(417.93), None, np.float64(188.9), np.float64(417.93), None, np.float64(188.9), np.float64(417.93), None, np.float64(188.9), np.float64(417.93), None, np.float64(417.93), np.float64(647.44), None, np.float64(417.93), np.float64(647.44), None, np.float64(417.93), np.float64(647.44), None, np.float64(417.93), np.float64(647.44), None, np.float64(417.93), np.float64(647.44), None, np.float64(417.93), np.float64(647.44), None, np.float64(417.93), np.float64(647.44), None, np.float64(417.93), np.float64(647.44), None, np.float64(647.44), np.float64(878.86), None, np.float64(647.44), np.float64(878.86), None, np.float64(647.44), np.float64(878.86), None, np.float64(647.44), np.float64(878.86), None, np.float64(647.44), np.float64(878.86), None, np.float64(647.44), np.float64(878.86), None, np.float64(647.44), np.float64(878.86), None, np.float64(647.44), np.float64(878.86), Non

In [101]:
# Визуализируем рёбра графа (основные линии)
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines',
)

# Визуализируем вершины графа
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=node_text,
    textposition="top center",
    hoverinfo='text',
    marker=dict(
        color='lightblue',
        size=10,
        line_width=2
    )
)

# Создаём фигуру
fig = go.Figure(
    data=[edge_trace, node_trace],
    layout=go.Layout(
        title='Scene Graph (Directed)',
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        height=1000, 
        width=1000
    )
)

# Добавляем стрелки для ориентации рёбер (исправленная версия)
def add_arrows(fig, edge_x, edge_y, arrow_length=0.1):
    arrows = []
    i = 0
    while i < len(edge_x):
        # Пропускаем None-значения (разрывы линий)
        if edge_x[i] is None or edge_y[i] is None:
            i += 1
            continue
            
        x0, y0 = edge_x[i], edge_y[i]    # Начало ребра
        
        # Ищем следующий не-None элемент (конец ребра)
        j = i + 1
        while j < len(edge_x) and (edge_x[j] is None or edge_y[j] is None):
            j += 1
            
        if j >= len(edge_x):
            break
            
        x1, y1 = edge_x[j], edge_y[j]  # Конец ребра
        
        # Вычисляем направление ребра
        dx = x1 - x0
        dy = y1 - y0
        length = np.sqrt(dx**2 + dy**2)
        
        if length == 0:
            i = j + 1
            continue
            
        # Нормализуем и укорачиваем стрелку
        dx = dx / length * (length - arrow_length)
        dy = dy / length * (length - arrow_length)
        
        # Добавляем стрелку как аннотацию
        fig.add_annotation(
            x=x0 + dx,
            y=y0 + dy,
            ax=x0,
            ay=y0,
            xref="x",
            yref="y",
            axref="x",
            ayref="y",
            showarrow=True,
            arrowhead=2,
            arrowsize=1,
            arrowwidth=1,
            arrowcolor='#888'
        )
        
        i = j + 1
    return fig

# Добавляем стрелки к фигуре
fig = add_arrows(fig, edge_x, edge_y)

fig.show()